In [33]:
import gzip
import time
import zlib
from seleniumwire import webdriver
import pandas as pd
import numpy as np
from ast import literal_eval
from pprint import pprint
from datetime import datetime

In [29]:
FOTMOB_BASE_URL = "https://www.fotmob.com"
season_url = "https://www.fotmob.com/leagues/9478/matches/indian-super-league"
# ISL_MAIN_URL = "https://www.scoresway.com/en_GB/soccer/indian-super-league-2023-2024/as2gkuxuvi5zqktyqlc2t4kk4/results"
# ISL_MAIN_URL = "https://www.scoresway.com/en_GB/soccer/indian-super-league-2024-2025/2h95ink16qa9eabhpeydbyz9w/results"

In [3]:
def getSoup(url) :
    res = requests.get(url)
    soup = BeautifulSoup(res.text)
    sleep(3)
    return soup

def getDriver(url) :
    driver = webdriver.Chrome()
    driver.get(url)
    driver.implicitly_wait(10)
    sleep(5)
    return driver  
    
# def removeCookieDialog(driver):
#     driver.execute_script("var obj = document.getElementById('onetrust-banner-sdk');if(obj){obj.remove()};")

def getMaxElement() :
    driver = getDriver(ISL_MAIN_URL)
    removeCookieDialog(driver)
    clicker = driver.find_element(By.CSS_SELECTOR, '.Opta-Exp')
    clicker.click()

    ul_element = driver.find_element(By.CSS_SELECTOR, 'ul.Opta-Cf')
    max_index = len(ul_element.find_elements(By.TAG_NAME, 'li'))
    driver.close()
    return max_index

In [4]:
max_index = getMaxElement()

In [1]:
# mw_result_links = {}
# backup_of_urls = []
# for i in range(max_index) :
#     inner_driver = getDriver(ISL_MAIN_URL)
#     removeCookieDialog(inner_driver)
#     clicker = inner_driver.find_element(By.CSS_SELECTOR, '.Opta-Exp')
#     clicker.click()
#     ul_element = inner_driver.find_element(By.CSS_SELECTOR, 'ul.Opta-Cf')
    
#     all_mw = ul_element.find_elements(By.TAG_NAME, 'a')
#     if len(all_mw) < i :
#         print(i,"Overflow",all_mw)
#     mw_item  = all_mw[i]
#     name = mw_item.text
#     print(i,"WORKING FOR : ",name,mw_item.tag_name)
#     # continue
#     mw_item.click()
#     fixture_tables =[x for x in inner_driver.find_elements(By.CSS_SELECTOR,'tbody.Opta-result.Opta-fixture') if x.text]
#     urls = []
#     for j,fixture in enumerate(fixture_tables) :
#         try :
#             print(f"Working on {name}, Game {j+1}")
#             match_page = fixture.find_element(By.CSS_SELECTOR, '[title="View match"]')
#             match_page.click()
#             # sleep(5)
#             removeCookieDialog(inner_driver)
#             # results_panes = inner_driver.find_elements(By.CSS_SELECTOR,'ul.striplist')
#             result_urls = [x.find_element(By.TAG_NAME,'a').get_property('href')
#                      for x in inner_driver.find_element(By.CSS_SELECTOR,'ul.striplist').find_elements(By.TAG_NAME,'li') 
#                      if x.text == 'PLAYER STATS']
#             if result_urls :
#                 urls.append((j,result_urls[0]))
#             inner_driver.execute_script("window.history.go(-1)")
#             # break
#         except Exception as e :
#             print(type(e),print(e))
#             break
    
#     if not name :
#         name = "Playoff"
#     if name not in mw_result_links :
#         mw_result_links[name] = urls
#     backup_of_urls.append(urls)
#     inner_driver.close()

In [85]:
def get_all_match_links(url):
    driver = webdriver.Chrome()  # Use your specific driver setup
    driver.get(url)
    
    # Wait for 30 seconds to allow all requests to complete
    time.sleep(30)
    data = {}
    match_lists = []
    for request in driver.requests:
        try:
            if f"{FOTMOB_BASE_URL}/api/leagues" in request.url :# \
            # or "https://api.performfeeds.com/soccerdata/squad" in request.url :
                print(request.url)
                if b'\x1f\x8b' in request.response.body[:2]:  # GZIP magic number
                    body = gzip.decompress(request.response.body)
                else:  # Assume Deflate
                    body= zlib.decompress(request.response.body, wbits=zlib.MAX_WBITS | 32)
                decoded_body = json.loads(body.decode())
                for match in decoded_body['matches']['allMatches']:
                    temp = dict()
                    temp['status'] = match.get('status',{}).get('finished',None)
                    temp['utc_time'] = match.get('status',{}).get('utcTime',None)
                    if temp['utc_time'] is not None :
                        if '.' in temp['utc_time']:
                            temp['utc_time'] = datetime.strptime(temp['utc_time'],"%Y-%m-%dT%H:%M:%S.%fZ")
                        else:
                            temp['utc_time'] = datetime.strptime(temp['utc_time'],"%Y-%m-%dT%H:%M:%SZ")
                    temp['home_name'] = match.get('home',{}).get('name',None)
                    temp['away_name'] = match.get('away',{}).get('name',None)
                    temp['page_url'] = f"{FOTMOB_BASE_URL}{match.get('pageUrl',None)}"
                    temp['match_id'] = match.get('id',None)
                    if temp['utc_time'] < datetime.now():
                        match_lists.append(temp)
        except Exception as e:
            print(f"Exception in {request.url}, some problme, fak it : {e}")
            print()
            print(body)
    driver.quit()
    return match_lists

In [86]:
match_lists = get_all_match_links(season_url)

https://www.fotmob.com/api/leagues?id=9478&newUefaBracket=true


In [99]:
match_lists[0]

{'status': True,
 'utc_time': datetime.datetime(2024, 9, 13, 14, 0),
 'home_name': 'Mohun Bagan SG',
 'away_name': 'Mumbai City FC',
 'page_url': 'https://www.fotmob.com/matches/mohun-bagan-sg-vs-mumbai-city-fc/8jna1366#4617934',
 'match_id': '4617934'}

In [91]:
url = match_lists[0]['page_url']

In [92]:
# def get_match_info(url) :
driver = webdriver.Chrome()  # Use your specific driver setup
driver.get(url)

# Wait for 30 seconds to allow all requests to complete
time.sleep(30)
data = {}
print(url)
for request in driver.requests:
    try:
        if f"{FOTMOB_BASE_URL}/api/matchDetails" in request.url :# \
        # or "https://api.performfeeds.com/soccerdata/squad" in request.url :
            print(request.url)
            if b'\x1f\x8b' in request.response.body[:2]:  # GZIP magic number
                body = gzip.decompress(request.response.body)
            else:  # Assume Deflate
                body= zlib.decompress(request.response.body, wbits=zlib.MAX_WBITS | 32)
        decoded_body = json.loads(body.decode())
        # for match in decoded_body:
        #     temp = dict()
        #     temp['status'] = match.get('status',{}).get('finished',None)
        #     temp['utc_time'] = match.get('status',{}).get('utcTime',None)
        #     temp['home_name'] = match.get('home',{}).get('name',None)
        #     temp['away_name'] = match.get('away',{}).get('name',None)
        #     temp['page_url'] = f"{FOTMOB_BASE_URL}{match.get('pageUrl',None)}"
        #     temp['match_id'] = match.get('id',None) 
        #     match_lists.append(temp)
            
    except Exception as e:
        print(f"Exception in {request.url}, some problme, fak it : {e}")
driver.quit()


https://www.fotmob.com/matches/mohun-bagan-sg-vs-mumbai-city-fc/8jna1366#4617934
https://www.fotmob.com/api/matchDetails?matchId=4617934&showNewUefaBracket=true


In [97]:
shots = decoded_body['content']['shotmap']['shots']

In [101]:
df = pd.read_excel(r"D:\research_locale\ISL Info\24-25\2024_09_13_Mohun_Bagan_Super_Giant_vs_Mumbai_City_FC.xlsx")

In [127]:
query = (df['typeId'] == 'Attempt Saved ') | (df['typeId'] == 'Miss ') | (df['typeId'] == 'Goal ')
shots_df = df[query]

In [128]:
shots[0]

{'id': 2719339057,
 'eventType': 'AttemptSaved',
 'teamId': 578655,
 'playerId': 570680,
 'playerName': 'Jon Toral',
 'x': 92.1,
 'y': 30.304,
 'min': 4,
 'minAdded': None,
 'isBlocked': True,
 'isOnTarget': True,
 'blockedX': 102.4396551726,
 'blockedY': 34.68625,
 'goalCrossedY': 36.21125,
 'goalCrossedZ': 0.1797894728,
 'expectedGoals': 0.04582624509930611,
 'expectedGoalsOnTarget': 0,
 'shotType': 'RightFoot',
 'situation': 'RegularPlay',
 'period': 'FirstHalf',
 'isOwnGoal': False,
 'onGoalShot': {'x': 0.41501322751322767,
  'y': 0.04756335259259259,
  'zoomRatio': 1},
 'isSavedOffLine': False,
 'isFromInsideBox': True,
 'firstName': 'Jon',
 'lastName': 'Toral',
 'fullName': 'Jon Toral',
 'teamColor': '#6CABDD'}

In [129]:
shots_df.iloc[0].to_dict()

{'id': 2719339057,
 'eventId': 40,
 'typeId': 'Attempt Saved ',
 'periodId': 1,
 'Minute': 3,
 'Seconds': 19,
 'Team': 'Mumbai City FC',
 'outcome': 'Successful',
 'x': 86.6,
 'y': 44.9,
 'timeStamp': '2024-09-13T14:04:20.582Z',
 'lastModified': '2024-09-13T14:05:01Z',
 'qualifier': "[{'id': 5205747595, 'qualifierId': 139, 'qualifierName': 'Own Player', 'description': 'Shot saved by goalkeeper that was deflected by a defender'}, {'id': 5205747583, 'qualifierId': 154, 'qualifierName': 'Intentional assist', 'description': 'Shot from an intentional assist i.e. The assisting player intended the pass, no deflection etc'}, {'id': 5205747597, 'qualifierId': 100, 'qualifierName': 'Six yard blocked', 'description': 'Shot blocked on the 6 yard line'}, {'id': 5205747593, 'qualifierId': 328, 'qualifierName': 'NA', 'description': 'NA'}, {'id': 5205747585, 'qualifierId': 55, 'value': '39', 'qualifierName': 'Related event ID', 'description': 'This will appear for goals or shots, the related event_id 

In [122]:
p = []
for i in range(len(shots)):
    temp = {}
    # fotmob
    fm_shot = shots[i]
    temp['f_type'] = fm_shot.get('eventType')
    temp['f_x'] = fm_shot.get('x')
    temp['f_y'] = fm_shot.get('y')
    temp['f_min'] = fm_shot.get('min')
    # sa
    sa_shot = shots_df.iloc[i].to_dict()
    temp['sa_type'] = sa_shot.get('typeId')
    temp['sa_x'] = sa_shot.get('x')
    temp['sa_y'] = sa_shot.get('y')
    temp['sa_min'] = sa_shot.get('Minute')
    temp['sa_sec'] = sa_shot.get('Seconds')
    p.append(temp)

In [125]:
pd.DataFrame(p).to_csv("Example.csv")